Сделано на основе https://colab.sandbox.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l10c04_nlp_optimizing_the_text_generation_model.ipynb

## Import TensorFlow and related functions

In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Other imports for processing data
import string
import numpy as np
import pandas as pd

Может обучить сначала на куче текстов, а потом трансформнуть для сектора

In [ ]:
!wget https://github.com/d0rj/RusLit/archive/refs/heads/main.zip --output-document ./literature.zip

In [ ]:
!unzip literature.zip

In [2]:
import os
raw_literature = []
for (root,dirs,files) in os.walk('./RusLit', topdown=True):
  for f in files:
    f_name = os.path.join(root, f)
    if f_name.endswith('.txt'):
      x = 100
      #print(f_name)
      with open(f_name, 'r', encoding='utf-8') as file:
        try:
          paragraph = file.read()
          raw_literature.extend([paragraph[i: i + x] for i in range(0, len(paragraph), x)])
        except:
          pass

In [3]:
print(len(raw_literature))
raw_literature_2 = list(map(lambda x: x.replace('\n\n', ' \n'), raw_literature))
#print(songs[0])
raw_literature_2 = list(map(lambda x: x.replace('\n', ' endline\n'), raw_literature_2))
#print(songs[0])
print(raw_literature_2[:10])
dataset = pd.DataFrame(raw_literature_2[:10_000], columns=['text'])

392337
['Александр Блок  endline\n endline\nВОЗМЕЗДИЕ  endline\n endline\n                                                   Юность - это возмезд', 'ие. endline\n                                                                 Ибсен  endline\nПРЕДИСЛОВИЕ   endline\n      endline\n    ', ' Не  чувствуя ни нужды, ни охоты заканчивать поэму, полную революционных endline\nпредчувствий,  в  года,  ко', 'гда  революция  уже произошла, я хочу предпослать endline\nнаброску  последней  главы  рассказ  о  том, как п', 'оэма родилась, каковы были endline\nпричины ее возникновения, откуда произошли ее ритмы.  endline\n     Интересно  и  ', 'небесполезно  и для себя, и для других припомнить историю endline\nсобственного  произведения. К тому же нам,', ' счастливейшим или несчастливейшим endline\nдетям  своего  века,  приходится помнить всю свою жизнь; все годы', ' наши резко endline\nокрашены  для  нас,  и  -  увы!  -  забыть их нельзя, - они окрашены слишком endline\nнеизгладимо', ',  так  что каж

Нет! Мы нафигачим сюда текстов Сектора газа!

In [4]:
from get_sector_songs import get_sector_songs
dataset_2 = get_sector_songs()

### Preprocessing

In [5]:
def tokenize_corpus(corpus, num_words=-1):
  # Fit a Tokenizer on the corpus
  if num_words > -1:
    tokenizer = Tokenizer(num_words=num_words)
  else:
    tokenizer = Tokenizer()
  tokenizer.fit_on_texts(corpus)
  return tokenizer

def create_lyrics_corpus(dataset, field):
  # Remove all other punctuation
  dataset[field] = dataset[field].str.replace('[{}]'.format(string.punctuation), '')
  # Make it lowercase
  dataset[field] = dataset[field].str.lower()
  # Make it one long string to split by line
  lyrics = dataset[field].str.cat()
  corpus = lyrics.split('\n')
  # Remove any trailing whitespace
  for l in range(len(corpus)):
    corpus[l] = corpus[l].rstrip()
  # Remove any empty lines
  corpus = [l for l in corpus if l != '']

  return corpus

def tokenize_corpus(corpus, num_words=-1):
  # Fit a Tokenizer on the corpus
  if num_words > -1:
    tokenizer = Tokenizer(num_words=num_words)
  else:
    tokenizer = Tokenizer()
  tokenizer.fit_on_texts(corpus)
  return tokenizer

# Read the dataset from csv - this time with 250 songs
#dataset = pd.read_csv('/tmp/songdata.csv', dtype=str)[:250]
# Create the corpus using the 'text' column containing lyrics
def create_imput_data(corpus, max_sequence_len=None):
  sequences = []
  for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      sequences.append(n_gram_sequence)

  # Pad sequences for equal input length 
  if max_sequence_len is None:
    max_sequence_len = max([len(seq) for seq in sequences])
  sequences = np.array(pad_sequences(sequences, maxlen=max_sequence_len, padding='pre'))

  # Split sequences between the "input" sequence and "output" predicted word
  input_sequences, labels = sequences[:,:-1], sequences[:,-1]
  # One-hot encode the labels
  one_hot_labels = tf.keras.utils.to_categorical(labels, num_classes=total_words)
  return max_sequence_len, input_sequences, one_hot_labels

corpus = create_lyrics_corpus(dataset, 'text')
# Tokenize the corpus
tokenizer = tokenize_corpus(corpus, num_words=6000)
total_words = tokenizer.num_words

# There should be a lot more words now
print(total_words)

max_sequence_len, input_sequences, one_hot_labels = create_imput_data(corpus)
_, input_sequences_2, one_hot_labels_2 = create_imput_data(create_lyrics_corpus(dataset_2, 'text'), max_sequence_len)
print(input_sequences.shape)
print(input_sequences_2.shape)

6000
(96369, 13)
(2043, 13)


### Train a (Better) Text Generation Model

Сначала Корпус русской классики

In [6]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, AdditiveAttention

model = Sequential()
model.add(Embedding(total_words, 128, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(total_words, activation='softmax'))
opt = tf.keras.optimizers.SGD(learning_rate=0.1, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(input_sequences, one_hot_labels, epochs=5, verbose=1)

Epoch 1/5
3012/3012 [==============================] - 72s 22ms/step - loss: 5.8602 - accuracy: 0.2916
Epoch 2/5
3012/3012 [==============================] - 69s 23ms/step - loss: 5.4459 - accuracy: 0.2943
Epoch 3/5
2221/3012 [=====================>........] - ETA: 25s - loss: 5.2319 - accuracy: 0.2976

## Трансфер

In [ ]:
model.save('fitsrt_step_model') # saves compiled state

In [ ]:
model2 = tf.keras.models.load_model('fitsrt_step_model')
history2 = model2.fit(input_sequences_2, one_hot_labels_2, epochs=20, verbose=1)

### View the Training Graph

In [ ]:
import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

plot_graphs(history, 'accuracy')

### Varying the Possible Outputs

In running the above, you may notice that the same seed text will generate similar outputs. This is because the code is currently always choosing the top predicted class as the next word. What if you wanted more variance in the output? 

Switching from `model.predict_classes` to `model.predict_proba` will get us all of the class probabilities. We can combine this with `np.random.choice` to select a given predicted output based on a probability, thereby giving a bit more randomness to our outputs.

In [ ]:
# Use this process for the full output generation
seed_text = "Грустный клоун"
next_words = 100
  
for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted_probs = model.predict(token_list, verbose=0)[0]
  predicted = np.random.choice([x for x in range(len(predicted_probs))],
                               p=predicted_probs)
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += " " + output_word
seed_text = seed_text.replace('endline', '\n')
print(seed_text)

In [ ]:
# Use this process for the full output generation
seed_text = "Грустный клоун"
next_words = 100
  
for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted_probs = model2.predict(token_list, verbose=0)[0]
  predicted = np.random.choice([x for x in range(len(predicted_probs))],
                               p=predicted_probs)
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += " " + output_word
seed_text = seed_text.replace('endline', '\n')
print(seed_text)